<a href="https://colab.research.google.com/github/minhaz1172/Deep-Learning-with-PyTorch/blob/main/Building_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# upload kaggle.json()

In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"minhaz1172","key":"c20c43ca9e6e5be6fa69c912fab503c3"}'}

# check GPU Availability

In [2]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

# Move and Set Permission

In [3]:
import os
import zipfile

# Make directory and move kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/

# Set proper permissions
!chmod 600 ~/.kaggle/kaggle.json

# install kaggle package

In [4]:
!pip install -q kaggle

# find and download dataset

In [5]:
!kaggle datasets download -d zalando-research/fashionmnist

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
  0% 0.00/68.8M [00:00<?, ?B/s]
100% 68.8M/68.8M [00:00<00:00, 971MB/s]


# unzip the dataset

In [6]:
!unzip /content/fashionmnist.zip

Archive:  /content/fashionmnist.zip
  inflating: fashion-mnist_test.csv  
  inflating: fashion-mnist_train.csv  
  inflating: t10k-images-idx3-ubyte  
  inflating: t10k-labels-idx1-ubyte  
  inflating: train-images-idx3-ubyte  
  inflating: train-labels-idx1-ubyte  


# import necessRY LIABRARIES

In [7]:
import numpy as np
import pandas as pd
import torch

Fashion-MNIST is a dataset of Zalando's article images—consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes. Zalando intends Fashion-MNIST to serve as a direct drop-in replacement for the original MNIST dataset for benchmarking machine learning algorithms

In [8]:
# SET random seeds for reproducibility
torch.manual_seed(42)

# Load csv fi;e

In [9]:
df=pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# sepate feature and values
# iloc[:, 1:] = select: ,All rows (:),From column 1 to the last column (1:), skipping column 0,These columns represent the 784 pixel values of each image (flattened),
#.values:
#Converts the selected DataFrame part to a NumPy array,So X will have shape: (60000, 784) → 60,000 images, 784 features each

In [10]:
X=df.iloc[:,1:].values
y=df.iloc[:,0].values

# train test split

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

# Scaling Features
Each pixel value ranges from 0 to 255 (grayscale)

But neural networks work better when inputs are scaled between 0 and 1

So dividing by 255 does:

0 → 0.0

255 → 1.0

128 → 0.50196 ... and so on.

This converts all pixel values into the [0, 1] range.

# Now why reshape in CNN:
CNNs expect 4D input tensors:

CNNs expect 4D input tensors:
(N, C, H, W) = (batch_size, channels, height, width)
So:

For grayscale images (like MNIST): C = 1

For RGB images: C = 3

In [12]:
X_train = X_train / 255.0          # Normalization
X_test = X_test / 255.0            # Normalization

X_train = X_train.reshape(-1, 1, 28, 28)  # Reshape for CNN input
X_test = X_test.reshape(-1, 1, 28, 28)


# Dataset Class

In [13]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)  # normalize already done
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.features)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

create a train dataset object

In [14]:
train_dataset=CustomDataset(X_train,y_train)

# Len of train dataset

In [15]:
len(train_dataset)

48000

# create test dataset

In [16]:
test_dataset=CustomDataset(X_test,y_test)

# create train and test loader

In [17]:
train_loader=DataLoader(train_dataset,batch_size=32,shuffle=True,pin_memory=True)
test_loader=DataLoader(test_dataset,batch_size=32,shuffle=False,pin_memory=True)

In [18]:
X_train.shape[1]

1

# Define CNN

 # nn.BatchNorm2d(32)
  #32 = number of channels (feature maps), not batch size!

  # It normalizes each of the 32 channels independently over the batch.

In [19]:
import torch.nn as nn
class CNN(nn.Module):
  def __init__(self,input_channel):
    super().__init__()

    self.model=nn.Sequential(
        nn.Conv2d(input_channel,32,kernel_size=3,padding=1), # 32 is filter
        # Output = [(Input - Kernel + 2 × Padding) / Stride] + 1
        ## Output: (32, 28, 28)
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2), # Output = [(Input - Kernel) / Stride] + 1
        ## Output: (32, 14, 14)

        nn.Conv2d(32,64,kernel_size=3,padding=1),  # (64, 14, 14)
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),  # (64, 7, 7)


        nn.Conv2d(64,128,kernel_size=3,padding=1),  # (128, 7, 7)
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,stride=2),  # (64,3,3) # 3.5=>3

       # fully connected layer
        nn.Flatten(),                           # Flattens input from (128, 3, 3) => (1152)
        nn.Linear(128 * 3 * 3, 128),            # Fully connected layer
        nn.ReLU(),                              # Activation
        nn.Dropout(0.4),                        # Regularization

        nn.BatchNorm1d(128),                   # Batch normalization for 1D (FC) features

       nn.Linear(128, 64),                     # Second FC layer
       nn.ReLU(),
       nn.Dropout(0.2),                        # Dropout again

       nn.Linear(64, 10)                       # Final output layer (e.g., for 10-class classification)


    )

  def forward(self,x):
    return self.model(x)

In [20]:
learning_rate=0.1
epochs=100

In [21]:
model=CNN(1) # sending number of chaneels of images===example if rgb channel =3\\
model.to(device)


CNN(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=1152, out_features=128, bias=True)
    (14): ReLU()
    (15): Dropout(p=0.4, inplace=False)
    (16): Bat

# loos and optimizer

In [22]:
import torch.optim as optim
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(model.parameters(),lr=learning_rate,weight_decay=1e-4)

# training Loop

In [24]:
for epoch in range(epochs):
  total_epoch_loss=0

  for features,labels in train_loader:

    # move to gpu
    features,labels=features.to(device),labels.to(device)

    #forward pass

    outputs=model(features) # it call=> model.__call__(features) → model.forward(features)



    # calculate loss
    loss=criterion(outputs,labels)

    # backpass

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


    total_epoch_loss=total_epoch_loss+loss.item()

    avg_loss=total_epoch_loss/len(train_loader)

    print(f"Epoch:{epoch+1},Loss:{avg_loss:.4f}")





Streaming output truncated to the last 5000 lines.
Epoch:97,Loss:0.0988
Epoch:97,Loss:0.0989
Epoch:97,Loss:0.0990
Epoch:97,Loss:0.0991
Epoch:97,Loss:0.0993
Epoch:97,Loss:0.0994
Epoch:97,Loss:0.0995
Epoch:97,Loss:0.0995
Epoch:97,Loss:0.0996
Epoch:97,Loss:0.0997
Epoch:97,Loss:0.0998
Epoch:97,Loss:0.0999
Epoch:97,Loss:0.1000
Epoch:97,Loss:0.1001
Epoch:97,Loss:0.1001
Epoch:97,Loss:0.1002
Epoch:97,Loss:0.1002
Epoch:97,Loss:0.1003
Epoch:97,Loss:0.1006
Epoch:97,Loss:0.1006
Epoch:97,Loss:0.1007
Epoch:97,Loss:0.1008
Epoch:97,Loss:0.1008
Epoch:97,Loss:0.1009
Epoch:97,Loss:0.1009
Epoch:97,Loss:0.1010
Epoch:97,Loss:0.1010
Epoch:97,Loss:0.1010
Epoch:97,Loss:0.1011
Epoch:97,Loss:0.1012
Epoch:97,Loss:0.1012
Epoch:97,Loss:0.1015
Epoch:97,Loss:0.1015
Epoch:97,Loss:0.1016
Epoch:97,Loss:0.1016
Epoch:97,Loss:0.1017
Epoch:97,Loss:0.1019
Epoch:97,Loss:0.1020
Epoch:97,Loss:0.1023
Epoch:97,Loss:0.1024
Epoch:97,Loss:0.1024
Epoch:97,Loss:0.1027
Epoch:97,Loss:0.1028
Epoch:97,Loss:0.1029
Epoch:97,Loss:0.1029
Epoc

# set model On valuation

In [25]:
# set model on evaluation mode
model.eval()

CNN(
  (model): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (12): Flatten(start_dim=1, end_dim=-1)
    (13): Linear(in_features=1152, out_features=128, bias=True)
    (14): ReLU()
    (15): Dropout(p=0.4, inplace=False)
    (16): Bat

In [27]:

total=0
correct=0

with torch.no_grad():
  for features,labels in test_loader:
     features,labels=features.to(device),labels.to(device)
     y_pred=model(features)
     _,predicted=torch.max(y_pred.data,1)
     total=total+labels.shape[0]
     correct=correct+(predicted==labels).sum().item()
     accuracy=100*correct/total
print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 91.67%
